In [ ]:
import json
import os
os.environ["PATH"] += os.pathsep + "/YOUR/ENVIRONMENT"

import whisper
import torch

In [2]:
def use_model(model_folder:str, model:str) -> str:
    """
    Ensures that a specified Whisper model is available for use. If the model does not exist
    in the specified folder, it downloads the model to that location.

    Parameters
    ----------
    model_folder : str
        The folder where the model should be saved or checked for existence.
    model : str
        The name of the model to use (e.g., "small", "medium").

    Returns
    -------
    str
        The full path to the model file.

    Raises
    ------
    Exception
        If there is an error during model download or validation.
    """
    
    try:
        # Construct the full model path
        model_path = os.path.join(model_folder, model + '.pt')
        
        # Check of the model already exists
        if os.path.exists(model_path):
            print(f"You are using {model_path}")
            return model_path

        # If the model doesn't exist, prompt the user to confirm the model name
        print(f"Available models: {whisper.available_models()}")
        model_input = input("Please provide the name of the model to download: ").strip()

        # Validate the provided model name
        if model_input not in whisper.available_models():
            raise ValueError(f"Invalid model name. Available models are: {whisper.available_models()}")

        # Check again if the input model is already downloaded
        model_path = os.path.join(model_folder, model_input + '.pt')
        if os.path.exists(model_path):
            print(f"Model '{model_input}' is already downloaded and available at {model_path}")
            return model_path
            
        # Download the model
        whisper.load_model(model_input, download_root=model_folder)
        print(f"Model '{model_input}' downloaded successfully to {model_path}")
        return model_path
            
    except Exception as e:
        print(f"An error occurred while downloading the model: {e}")
        raise  # Re-raise the exception for upstream handling

In [1]:
def transcribe(model_folder: str, model:str, audio: str, output_folder: str) -> None:
    """
    This function loads a Whisper model from a specified file path, uses it to transcribe
    a given audio file, and then saves the transcription to a text file in the specified output folder.
    
    Parameters
    ----------
    model_path : str
        The file path to the Whisper model (e.g., "medium.pt").
    audio : str
        The file path to the audio file to be transcribed (e.g., "recording.m4a").
    output_folder : str
        The directory path where the resulting transcription file will be saved.
    
    Returns
    -------
    None
        The function does not return a value, but it writes the transcription to a file.
    """

    # If a CUDA-compatible GPU is available, the transcription will be executed on the GPU for faster processing.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {'GPU: ' + torch.cuda.get_device_name(0) if device.type == 'cuda' else 'CPU'}")

    # Load the Whisper model from the given model_path and move it to the selected device (GPU/CPU).
    whisper_model = whisper.load_model(use_model(model_folder, model), device=device)

    # Transcribe the given audio file using the loaded Whisper model.
    result = whisper_model.transcribe(audio)

    # Construct the output file path for the transcription:
    file_name = os.path.basename(audio).split('.')[0]
    transcription_folder = os.path.join(output_folder, file_name)

    os.makedirs(transcription_folder, exist_ok=True)

    # Write the transcription data (result) to a JSON file.
    try:
        with open(os.path.join(transcription_folder, file_name + '.json'), "w") as fichier:
        	json.dump(result, fichier)
        print(f"Transcription data saved in : {transcription_folder, file_name + '.json'}")
    except Exception as e:
        print(f"Error saving JSON file: {e}")
    
    
    # Write the text transcription to a TXT file.
    try:
        with open(os.path.join(transcription_folder, file_name + '.txt'), 'w', encoding='utf-8') as file:
            file.write(result["text"])
        print(f"Transcription saved in : {transcription_folder, file_name + '.txt'}")
    except Exception as e:
        print(f"Error saving TXT file: {e}")

In [ ]:
# Path where your trained model is stored
model_folder = '/path/to/your/model/folder'

# Define the model variable
model_name = ''  # If left blank, the user will be prompted to choose a model

# Path to your audio file
audio = '/path/to/your/audio/file'

# Path where the output of your processing will be saved
output_folder = '/path/to/your/output/folder'

In [ ]:
%%prun
transcribe(model_folder, model, audio, output_folder)